# シーケンス分類器の訓練

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [19]:
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [8]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [21]:
n_epochs = 20     #100だと時間がかかるのでここでは20にしている
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.93333334 Test accuracy: 0.9084
1 Train accuracy: 0.96666664 Test accuracy: 0.9423
2 Train accuracy: 0.94 Test accuracy: 0.9503
3 Train accuracy: 0.9533333 Test accuracy: 0.9621
4 Train accuracy: 0.96666664 Test accuracy: 0.9607
5 Train accuracy: 0.9866667 Test accuracy: 0.9654
6 Train accuracy: 0.98 Test accuracy: 0.9595
7 Train accuracy: 0.9866667 Test accuracy: 0.9703
8 Train accuracy: 0.9866667 Test accuracy: 0.973
9 Train accuracy: 0.99333334 Test accuracy: 0.9679
10 Train accuracy: 0.97333336 Test accuracy: 0.9725
11 Train accuracy: 0.96666664 Test accuracy: 0.9729
12 Train accuracy: 1.0 Test accuracy: 0.9738
13 Train accuracy: 0.96 Test accuracy: 0.9726
14 Train accuracy: 0.99333334 Test accuracy: 0.9737
15 Train accuracy: 0.99333334 Test accuracy: 0.9769
16 Train accuracy: 1.0 Test accuracy: 0.9738
17 Train accuracy: 0.9533333 Test accuracy: 0.9776
18 Train accuracy: 0.99333334 Test accuracy: 0.975
19 Train accuracy: 0.9866667 Test accuracy: 0.9717


In [25]:
print(y_test[:20])

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [38]:
test = X_test[:20].reshape((-1, 28, 28))

In [35]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = logits.eval(feed_dict={X:test})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [41]:
y_pred

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 7, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])